# Imports

In [1]:
from sat import *
from px import *
from run_px_experiments import parse_output

import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

#PROBLEM_FILE = './toy_sat_problems/toy_sat1.cnf'
#SOLUTIONS_FILE = './toy_sat_problems/toy_sat_1_solutions.txt'

#PROBLEM_FILE = './toy_sat_problems/toy_sat2.cnf'
#SOLUTIONS_FILE = './toy_sat_problems/toy_sat_2_solutions.txt'

#PROBLEM_FILE = './toy_sat_problems/toy_sat3.cnf'
#SOLUTIONS_FILE = './toy_sat_problems/toy_sat_3_solutions.txt'

#PROBLEM_FILE = './toy_sat_problems/paper_example.cnf'
#SOLUTIONS_FILE = './toy_sat_problems/paper_example_solutions.txt'

PROBLEM_FILE = './sat_problems/trial_problems/rbcl_xits_08_UNSAT.cnf'
SOLUTIONS_FILE = './results/ubcsat_outputs/rbcl_xits_08_UNSAT.txt'

In [ ]:
def draw_graph(graph, node_size=400):
    nx.draw(graph, with_labels=True, node_color='white', node_size=node_size)
    ax = plt.gca() # to get the current axis
    ax.collections[0].set_edgecolor("#000000") 

# Load Problems / Solutions

In [2]:
%%time
sat = read_sat_problem(PROBLEM_FILE)

#print('Clause #, Variables')
#for i in range(len(sat.clauses)):
#    print(i, sat.clauses[i])

#print('\nVariable #, Clauses')
#sat.create_var_to_clause_dict()
#print(sat.var_to_clause_dict)

#print('\nUnique Variables')
#print(sat.unique_vars)

sat

Wall time: 575 ms


SAT(n = 1278, m = 68055, k = 9, name = 'rbcl_xits_08_UNSAT.cnf')

In [3]:
%%time
solutions = parse_output(SOLUTIONS_FILE)

print(solutions.shape)

p1 = solutions[6]
p2 = solutions[7]

p1_unsat = np.argwhere(~(sat.evaluate_solution(p1))).flatten()
p2_unsat = np.argwhere(~(sat.evaluate_solution(p2))).flatten()
print(f'P1 Unsat: {len(p1_unsat)}')
print(f'P2 Unsat: {len(p2_unsat)}')

print('')
print(f'P1 Score: {sat.score_solution(p1)}')
print(f'P2 Score: {sat.score_solution(p2)}')

assert(sat.m - sat.score_solution(p1) == len(p1_unsat))
assert(sat.m - sat.score_solution(p2) == len(p2_unsat))

(20, 1278)
P1 Unsat: 10
P2 Unsat: 8

P1 Score: 68045
P2 Score: 68047
Wall time: 1.13 s


In [ ]:
"""with open(SOLUTIONS_FILE) as solutions_file:
    solutions = [line.strip() for line in solutions_file.readlines()]
    
print(f'P1 Bitstring: {solutions[0]}')
print(f'P2 Bitstring: {solutions[1]}')
print('')

p1 = bitstring_to_int_array(solutions[0])
p2 = bitstring_to_int_array(solutions[1])
print(f'P1 Assignments: {p1}')
print(f'P2 Assignments: {p2}')
print('')

p1_unsat = np.argwhere(~(sat.evaluate_solution(p1))).flatten()
p2_unsat = np.argwhere(~(sat.evaluate_solution(p2))).flatten()
print(f'P1 Unsat Clauses: {p1_unsat}, {sat.clauses[p1_unsat]}')
print(f'P2 Unsat Clauses: {p2_unsat}, {sat.clauses[p2_unsat]}')
print('')

print(f'P1 Score: {sat.score_solution(p1)}')
print(f'P2 Score: {sat.score_solution(p2)}')"""

# PX

In [ ]:
pos = {
 1: (108, 115),
 2: (86, 78),
 3: (109, 137),
 4: (127, 70),
 5: (90, 61),
 6: (68, 104),
 7: (18, 60),
 8: (30, 104),
 9: (166, 24),
 10: (66, 64),
 11: (102, 24),
 12: (43, 42),
 13: (42, 68),
 14: (163, 70),
 15: (18, 39),
 16: (102, 49),
 17: (69, 38),
 18: (31, 81)
}

colors = []
for node in vig.nodes():
    if node in [7,13,15]:
        colors.append(-1)
    elif node in [12,18]:
        colors.append(-0.5)
    else:
        colors.append(0)
#colors = [1 if node in [7,13,15] else 0 for node in vig.nodes()]

nx.draw(vig, with_labels=True, node_color=colors, cmap='RdYlGn', vmin=-1.0, vmax=1.0, node_size=400, pos=pos)
ax = plt.gca() # to get the current axis
ax.collections[0].set_edgecolor("#000000") 

In [4]:
%%time
vig = get_vig(sat)

print(vig.number_of_nodes(), 
      vig.number_of_edges(), 
      nx.number_connected_components(vig))

#draw_graph(vig)

1278 46395 1
Wall time: 707 ms


In [5]:
%%time
decomposed_vig = decompose_vig(vig, p1, p2)

print(decomposed_vig.number_of_nodes(), 
      decomposed_vig.number_of_edges(), 
      nx.number_connected_components(decomposed_vig))

#draw_graph(decomposed_vig)

154 458 1
Wall time: 195 ms


In [6]:
%%time
new_solution = partition_crossover(sat, decomposed_vig, p1, p2, none_fill='p1', verbose=2)

print('')
print(f'New Solution: {new_solution}')
print(f'New Solution Score: {sat.score_solution(new_solution)}')
print(f'Remaining Unsatisfied Clauses: {sat.clauses[~sat.evaluate_solution(new_solution)]}')


New Solution: [  -1   -2    3 ... 1276 1277 1278]
New Solution Score: 68047
Remaining Unsatisfied Clauses: [{2, 163, 71, 140, 48, 117, 25, 186, 94}
 {97, 5, 166, 74, 143, 51, 120, 28, 189}
 {33, 194, 102, 10, 171, 79, 148, 56, 125}
 {34, 195, 103, 11, 172, 80, 149, 57, 126}
 {129, 37, 198, 106, 14, 175, 83, 152, 60}
 {131, 39, 200, 108, 16, 177, 85, 154, 62} {-239, 262} {-21, -20, -19}]
Wall time: 747 ms


# PX Prime on P1

In [7]:
%%time
decomposed_sat, iterations = decompose_problem(sat, p1, p2, p1_unsat, p2_unsat, init_method='p1', verbose=1)

print('')
print(decomposed_sat.clauses)
print(decomposed_sat.unique_vars)

decomposed_sat

[3,4] Length of Init SWAPC: 9
[5] Length of Init VAR: 60
[5] Length of No Common Variables VAR: 8
Length of sat_by_common: 67542
[6] Length of SWAPC: 3341
[7] Length of SWAPC: 98
[8] Length of VAR: 26
[6] Length of SWAPC: 8283
[7] Length of SWAPC: 163
[8] Length of VAR: 51
[6] Length of SWAPC: 9710
[7] Length of SWAPC: 232
[8] Length of VAR: 71
[6] Length of SWAPC: 10821
[7] Length of SWAPC: 269
[8] Length of VAR: 88
[6] Length of SWAPC: 11934
[7] Length of SWAPC: 312
[8] Length of VAR: 101
[6] Length of SWAPC: 11982
[7] Length of SWAPC: 334
[8] Length of VAR: 110
[6] Length of SWAPC: 12010
[7] Length of SWAPC: 344
[8] Length of VAR: 115
[6] Length of SWAPC: 12028
[7] Length of SWAPC: 352
[8] Length of VAR: 119
[6] Length of SWAPC: 12042
[7] Length of SWAPC: 358
[8] Length of VAR: 122
[6] Length of SWAPC: 12048
[7] Length of SWAPC: 358
[8] Length of VAR: 122
Loop ran 10 times

[{162} {2} {3, 95} {6} {191} {9, 147} {10} {11} {81} {128, 36} {60} {153}
 {85} {132, 109} {19, 203} {45, 183}

SAT(n = 122, m = 358, k = 3, name = 'decomposed_rbcl_xits_08_UNSAT.cnf')

In [8]:
%%time
vig_prime = get_vig(decomposed_sat, verbose=False)

print(vig_prime.number_of_nodes(), 
      vig_prime.number_of_edges(), 
      nx.number_connected_components(vig_prime))

#draw_graph(vig_prime)

122 142 3
Wall time: 2.03 ms


In [9]:
%%time
decomposed_vig_prime = decompose_vig(vig_prime, p1, p2)

print(decomposed_vig_prime.number_of_nodes(), 
      decomposed_vig_prime.number_of_edges(), 
      nx.number_connected_components(decomposed_vig_prime))

#draw_graph(decomposed_vig_prime)

122 142 3
Wall time: 2.58 ms


In [10]:
%%time
new_solution_prime = partition_crossover(decomposed_sat, decomposed_vig_prime, p1, p2, none_fill='p1', verbose=2)

print('')
print(f'New Solution: {new_solution_prime}')
print(f'New Solution Score: {sat.score_solution(new_solution_prime)}')
print(f'Remaining Unsatisfied Clauses: {sat.clauses[~sat.evaluate_solution(new_solution_prime)]}')

Common variable assignments: [-1 None None ... None 1277 1278]
Component: {2, 3, 6, 9, 10, 11, 19, 36, 45, 95, 109, 128, 132, 147, 153, 162, 183, 191, 203, 209, 210, 213, 216, 217, 218, 226, 231, 232, 233, 236, 239, 240, 241, 249, 254, 255, 256, 259, 263, 264, 266, 272, 275, 277, 278, 279, 282, 286, 287, 289, 295, 298, 300, 301, 302, 305, 306, 309, 310, 312, 318, 321, 323, 324, 328, 329, 332, 333, 335, 339, 341, 344, 346, 347, 351, 352, 355, 356, 364, 367, 369, 370, 374, 375, 377, 378, 379, 387, 390, 393, 397, 398, 400, 401, 402, 410}
	Sub problem clauses: [{162} {2} {3, 95} {6} {191} {9, 147} {10} {11} {128, 36} {153} {132, 109}
 {19, 203} {45, 183} {209, -2} {2, -209} {210, -3} {3, -210} {-6, 213}
 {-213, 6} {216, -9} {-216, 9} {217, -10} {10, -217} {218, -11} {11, -218}
 {-11, -10, -9} {-95, -109} {226, -19} {19, -226} {-95, -109} {-95, -109}
 {-95, -109} {-95, -109} {-191, -203} {-95, -302} {-191, -203}
 {-191, -203} {-191, -203} {-191, -203} {-128, -335} {-132, -339}
 {-191, -203}

In [ ]:
old_method_unsat = set(np.argwhere(~sat.evaluate_solution(new_solution)).flatten().tolist())
new_method_unsat = set(np.argwhere(~sat.evaluate_solution(new_solution_prime)).flatten().tolist())
clauses_of_interest = list(new_method_unsat - old_method_unsat)
print(clauses_of_interest)
print(sat.clauses[clauses_of_interest])

vars_of_interest = get_unique_vars(sat.clauses[clauses_of_interest])
print(vars_of_interest)

print(f'P1  Assignments: {get_assignments(p1, vars_of_interest)}')
print(f'P2  Assignments: {get_assignments(p2, vars_of_interest)}')
print(f'NS  Assignments: {get_assignments(new_solution, vars_of_interest)}')
print(f'NS* Assignments: {get_assignments(new_solution_prime, vars_of_interest)}')

In [ ]:
get_unique_vars(sat.clauses[list(sat.clauses_with_variables([2865]))])

# PX Prime on P2

In [ ]:
%%time
decomposed_sat, iterations = decompose_problem(sat, p1, p2, p1_unsat, p2_unsat, init_method='p2', verbose=1)

#print('')
#print(decomposed_sat.clauses)
#print(decomposed_sat.unique_vars)

decomposed_sat

In [ ]:
%%time
vig_prime = get_vig(decomposed_sat, verbose=False)

print(vig_prime.number_of_nodes(), 
      vig_prime.number_of_edges(), 
      nx.number_connected_components(vig_prime))

#draw_graph(vig_prime)

In [ ]:
%%time
decomposed_vig_prime = decompose_vig(vig_prime, p1, p2)

print(decomposed_vig_prime.number_of_nodes(), 
      decomposed_vig_prime.number_of_edges(), 
      nx.number_connected_components(decomposed_vig_prime))

#draw_graph(decomposed_vig_prime)

In [ ]:
%%time
new_solution_prime = partition_crossover(decomposed_sat, decomposed_vig_prime, p1, p2, none_fill='p2', verbose=2)

print('')
print(f'New Solution: {new_solution_prime}')
print(f'New Solution Score: {sat.score_solution(new_solution_prime)}')
print(f'Remaining Unsatisfied Clauses: {sat.clauses[~sat.evaluate_solution(new_solution_prime)]}')

# PX Prime on XOR

In [ ]:
%%time
decomposed_sat, iterations = decompose_problem(sat, p1, p2, p1_unsat, p2_unsat, init_method='xor', verbose=1)

#print('')
#print(decomposed_sat.clauses)
#print(decomposed_sat.unique_vars)

decomposed_sat

In [ ]:
%%time
vig_prime = get_vig(decomposed_sat, verbose=False)

print(vig_prime.number_of_nodes(), 
      vig_prime.number_of_edges(), 
      nx.number_connected_components(vig_prime))

#draw_graph(vig_prime)

In [ ]:
%%time
decomposed_vig_prime = decompose_vig(vig_prime, p1, p2)

print(decomposed_vig_prime.number_of_nodes(), 
      decomposed_vig_prime.number_of_edges(), 
      nx.number_connected_components(decomposed_vig_prime))

#draw_graph(decomposed_vig_prime)

In [ ]:
%%time
new_solution_prime = partition_crossover(decomposed_sat, decomposed_vig_prime, p1, p2, none_fill='p1', verbose=2)

print('')
print(f'New Solution: {new_solution_prime}')
print(f'New Solution Score: {sat.score_solution(new_solution_prime)}')
print(f'Remaining Unsatisfied Clauses: {sat.clauses[~sat.evaluate_solution(new_solution_prime)]}')